In [ ]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(documents, columns=['course', 'section', 'question', 'text'])
df.head()

In [ ]:
df[df.course == 'data-engineering-zoomcamp'].head()